# Appending columns of case counts for new dates, to pre-existing zip code layers (points and polygons), for the Covid-19 dashboard. Sample.


Steps:

1. Retrieve the current layer from SDCounty dashboard, convert to a DF, convert to 4326, and transform to a DF by zip codes (using Pivot), resulting in a wide table

2. Save as a CSV file, making sure that **zipcode** in this file is a text column. Then add this file to AGOL. This will be the **source** file. 

3. Reference pre-existing zip code layers, both point and polygon. These are the **target** layers. These layers shall have a **ZIP** column, that will be used for joining data from the source. These layers should also have non-editable view layers, to be referenced from the dashboard. 

4. Figure out which fields from the source layer to add to the target layer, and then join them



In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
import pandas as pd
import json
from copy import deepcopy
import time
gis = GIS(username='xij080_ucsd5')  # this will ask for password. 
arcgis.__version__

Enter password: ········


'1.8.0'

## 1. Retrieve the current layer from SDCounty dashboard, convert to a DF, convert to 4326, and transform to a DF by zip codes (using Pivot), resulting in a wide table



In [2]:
sd_dashboard_service = 'https://services1.arcgis.com/1vIhDJwtG5eNmiqX/ArcGIS/rest/services/CovidDashUpdate/FeatureServer'
db_item = FeatureLayerCollection(sd_dashboard_service)
ZipsCompiled_df = pd.DataFrame.spatial.from_layer(db_item.layers[0])
# ConfirmHopsitalICuDeaths_df = pd.DataFrame.spatial.from_layer(db_item.layers[1])
# AgeGenderPoints_df = pd.DataFrame.spatial.from_layer(db_item.layers[2])
# CompiledCopyDashUpdate_df = pd.DataFrame.spatial.from_layer(db_item.layers[3])
# db_item.layers[0]

In [3]:
# convert the layer with counts by zips into a df, in 4326

ZipsCompiled_df_4326 = db_item.layers[0].query(out_sr = 4326, return_geometry=True).sdf


In [4]:
ZipsCompiled_df_4326.head()

,OBJECTID,SDEP_SANGIS_ZIPCODE_ZIP,ZipText,Zip_Code,Case_Count,UpdateDate,rate_100k,SHAPE
0,227,91901,91901,None,1.0,2020-04-01 08:00:00,None,"{""x"": -116.69553702167843, ""y"": 32.80570970685..."
1,228,91902,91902,91902,9.0,2020-04-01 08:00:00,None,"{""x"": -117.01506766053788, ""y"": 32.67158293810..."
2,229,91910,91910,91910,23.0,2020-04-01 08:00:00,None,"{""x"": -117.0656532926161, ""y"": 32.636413417033..."
3,230,91911,91911,91911,21.0,2020-04-01 08:00:00,None,"{""x"": -117.05021424270026, ""y"": 32.60730912252..."
4,231,91913,91913,91913,20.0,2020-04-01 08:00:00,None,"{""x"": -116.98749464783407, ""y"": 32.61626743309..."


In [5]:
# create the X and Y columns in this df. There is probably a better way to do this

for idx, row in ZipsCompiled_df_4326.iterrows():
    
    a = str(ZipsCompiled_df_4326.loc[idx,'SHAPE']).replace("'", '"')
    js = json.loads(a)
    ZipsCompiled_df_4326.loc[idx,'X'] =  js['x']
    ZipsCompiled_df_4326.loc[idx,'Y'] =  js['y']



In [6]:
# save only necessary columns into a csv file ZipsCompiled_df_4326_timestamp.csv, for service update. This is a complete current file

import csv
ts = time.gmtime()
currentfile = 'ZipsCompiled_df_4326_'+time.strftime("%Y%m%d_%H_%M_%S", ts) +'.csv'


# a really hacky way to force ZipText to be a string...

ZipsCompiled_df_4326_for_csv = ZipsCompiled_df_4326.copy()

ZipsCompiled_df_4326_for_csv['ZipText'] = ZipsCompiled_df_4326_for_csv['ZipText'].astype(str)

aa = ZipsCompiled_df_4326_for_csv.loc[0]['ZipText']
bb = "'" + aa + "'"
ZipsCompiled_df_4326_for_csv.loc[0,'ZipText'] = bb

ZipsCompiled_df_4326_for_csv.to_csv(currentfile,columns=['ZipText','Case_Count','UpdateDate','X','Y'], quotechar='"', quoting=csv.QUOTE_MINIMAL, index=False)

### Now, create a wide table and rename columns
    

In [7]:
# generate better time stamps
ZipsCompiled_df_4326['DateString'] = ZipsCompiled_df_4326['UpdateDate']
ZipsCompiled_df_4326['DateString'] = ZipsCompiled_df_4326['DateString'].dt.strftime("%m/%d/%y")
ZipsCompiled_df_4326=ZipsCompiled_df_4326.drop_duplicates(['ZipText','DateString'])

In [8]:
# generate a wide table for the current file. This will be a source datatable for updating zip code layers
zips_wide_df = ZipsCompiled_df_4326.pivot(index='ZipText', columns='DateString', values='Case_Count')

In [9]:
# zips_wide_df['zipcode'] = zips_wide_df['ZipText']
zips_wide_df['zipcode'] = zips_wide_df.index
#zips_wide_df.reset_index(inplace=True)
zips_wide_df.head(3)

DateString,03/30/20,03/31/20,04/01/20,04/02/20,04/03/20,04/04/20,04/05/20,04/06/20,04/07/20,04/08/20,...,07/21/20,07/22/20,07/23/20,07/24/20,07/25/20,07/26/20,07/27/20,07/28/20,07/29/20,zipcode
ZipText,,,,,,,,,,,,,,,,,,,,,
91901,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,55.0,55.0,56.0,57.0,60.0,62.0,66.0,67.0,68.0,91901
91902,8.0,8.0,9.0,10.0,10.0,11.0,11.0,14.0,16.0,16.0,...,140.0,143.0,144.0,149.0,152.0,154.0,155.0,158.0,160.0,91902
91905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,91905
91906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,14.0,14.0,15.0,15.0,15.0,16.0,16.0,16.0,91906
91910,17.0,21.0,23.0,28.0,28.0,30.0,30.0,34.0,39.0,43.0,...,1014.0,1035.0,1058.0,1097.0,1111.0,1133.0,1155.0,1164.0,1179.0,91910
91911,13.0,16.0,21.0,24.0,26.0,29.0,29.0,35.0,38.0,47.0,...,1273.0,1297.0,1325.0,1357.0,1370.0,1392.0,1419.0,1418.0,1447.0,91911
91913,14.0,19.0,20.0,21.0,23.0,25.0,25.0,26.0,28.0,32.0,...,476.0,481.0,489.0,502.0,505.0,517.0,523.0,531.0,539.0,91913
91914,2.0,2.0,2.0,3.0,4.0,4.0,4.0,6.0,7.0,8.0,...,152.0,161.0,166.0,170.0,173.0,175.0,174.0,175.0,176.0,91914
91915,4.0,5.0,7.0,8.0,12.0,12.0,12.0,14.0,16.0,19.0,...,264.0,270.0,278.0,285.0,289.0,294.0,301.0,309.0,313.0,91915


## 2. Save as a CSV file, making sure that "zipcode" in this file is a text column. Then add this file to AGOL. This will be the source file. 

In [10]:
widefile = 'ZipsWide_'+time.strftime("%Y%m%d_%H_%M_%S", ts) +'.csv'

zips_wide_df_for_csv = zips_wide_df.copy()
aa = zips_wide_df_for_csv.loc['91901']['zipcode']
bb = "'" + aa + "'"
zips_wide_df_for_csv.loc['91901','zipcode'] = bb

zips_wide_df_for_csv['zipcode'][0] = '91901'
zips_wide_df_for_csv.to_csv(widefile,quotechar='"', quoting=csv.QUOTE_MINIMAL, index=False)


# add the current csv file to AGOL:
zip_wide_props = {'title':'Updated zips_wide just downloaded '+time.strftime("%Y%m%d_%H_%M_%S", ts)}
csv_wide_item = gis.content.add(item_properties=zip_wide_props, data=widefile)
csv_wide_item 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<Item title:"Updated zips_wide just downloaded 20200731_21_25_31" type:CSV owner:xij080_ucsd5>

In [11]:
# we'll need *zip_wide_csv_info* in the last step, when appending a new column
zip_wide_csv_info = gis.content.analyze(item=csv_wide_item.id, file_type='csv', location_type='none')
#check the last updated date 
zip_wide_csv_info['publishParameters']['layerInfo']['fields'][-2]

{'name': '07_29_20',
 'type': 'esriFieldTypeDouble',
 'alias': '07/29/20',
 'sqlType': 'sqlTypeFloat',
 'nullable': True,
 'editable': True,
 'domain': None,
 'defaultValue': None,
 'locationType': 'unknown'}

In [12]:
# list the fields in the updated file. Eventually, this will be needed to figure out which of these fields to add
csv_name=[]
csv_to_be_add=[]
for csv_field in zip_wide_csv_info['publishParameters']['layerInfo']['fields']:
    csv_name.append(csv_field['name'])
csv_name=[i.replace('_','/') for i in csv_name]
for i in zips_wide_df_for_csv.columns:
    if i not in csv_name:
        csv_to_be_add.append(i)
csv_to_be_add=[i.replace('/','_') for i in csv_to_be_add]
csv_name=[i.replace('/','_') for i in csv_name]
csv_name.remove('zipcode')

## 3. Reference pre-existing zip code layers, both point and polygon. 

These are the **target** feature layers. These layers shall have a **ZIP** column, that will be used for joining data from the source. These layers should also have non-editable view layers, to be referenced from the dashboard. 



In [24]:
zip_point_base_fl = gis.content.get('2c18a0ec72bb4a0d87dd19e8e5578ce1')
zip_poly_base_fl = gis.content.get('f595732fd52342108cca7a487087d5c5')

## 4. Figure out which fields from the source layer to add to the target layer, and then join them

In [14]:
# let's do it with a point layer, as a demonstration

# see which fields are already included in the target layer

zip_point_lyr = zip_point_base_fl.layers[0]
point_name=[]
point_field_list=[]
new_dates=[]

for field in zip_point_lyr.properties.fields:
    point_name.append(field['name'])

for i in csv_name:
    new='F'+i
    if new not in point_name:
        new_dates.append(new)
        dols_field = dict(deepcopy(zip_point_lyr.properties.fields[1]))
        dols_field['name'] = new
        dols_field['alias'] = i
        dols_field['type'] = "esriFieldTypeDouble"
        point_field_list.append(dols_field)
        
#  Update feature layer definition with the new field using the add_to_definition() method.
zip_point_lyr.manager.add_to_definition({"fields":point_field_list})

{'success': True}

In [53]:
# samething to do with polygon layer
# zip_poly_lyr = zip_poly_base_fl.layers[0]
# poly_name=[]
# poly_field_list=[]

# for field in zip_poly_lyr.properties.fields:
#     poly_name.append(field['name'])

# for i in csv_name:
#     if i not in poly_name:
#         dols_field = dict(deepcopy(zip_poly_lyr.properties.fields[1]))
#         dols_field['name'] = 'F'+i
#         dols_field['alias'] = i
#         dols_field['type'] = "esriFieldTypeDouble"
#         poly_field_list.append(dols_field)
# zip_poly_lyr.manager.add_to_definition({"fields":poly_field_list})         

{'success': True}

In [15]:
# append attribute values

# here: 
# ZIP is the column name in the target layer that is used for the join. 
#      It is also an index, as established in the previous cell.
#      It is a text column with field definition matching zipcode column in the source file
#  
# zipcode is the matching column in the source layer. 
#
# 07_01_20 is a column name in both source and target (Double, in both cases) 
#
# if an error occurs, just rerun the cell
implement=new_dates
for i in implement:
    i=i.replace('F','')
    new='F'+i
    zip_point_lyr.append(item_id=csv_wide_item.id,
                          upload_format = 'csv',
                          field_mappings = [{"name":new, "source":i},
                                            {"name":"ZIP", "source":"zipcode"}],
                          source_info = zip_wide_csv_info['publishParameters'],
                          upsert=True,
                          update_geometry=False,
                          append_fields=[new, "ZIP"],
                          skip_inserts=True,
                          upsert_matching_field="ZIP")
    print('done for '+ i)

zip_point_lyr.append(item_id=csv_wide_item.id,
                          upload_format = 'csv',
                          field_mappings = [{"name":'Current_Cases', "source":new_dates[-1].replace('F','')},
                                            {"name":"ZIP", "source":"zipcode"}],
                          source_info = zip_wide_csv_info['publishParameters'],
                          upsert=True,
                          update_geometry=False,
                          append_fields=['Current_Cases', "ZIP"],
                          skip_inserts=True,
                          upsert_matching_field="ZIP")
print('done for current cases')

done for 07_29_20


In [18]:
#append values for poly layer
# implement=new_dates
# for i in implement:
#     zip_poly_lyr.append(item_id=csv_wide_item.id,
#                           upload_format = 'csv',
#                           field_mappings = [{"name":i, "source":i},
#                                             {"name":"ziptext", "source":"zipcode"}],
#                           source_info = zip_wide_csv_info['publishParameters'],
#                           upsert=True,
#                           update_geometry=False,
#                           append_fields=[i, "ziptext"],
#                           skip_inserts=True,
#                           upsert_matching_field="ziptext")
#     print('done for '+ i)

done for 07_16_20
done for 07_17_20
done for 07_18_20
done for 07_19_20
done for 07_20_20


## 5. Create the line chart for last week cases

In [16]:
# ONLY RUN ONCE
# create the last week line chart feature service
# all_dates=pd.DataFrame.spatial.from_layer(zip_point_base_fl.layers[0])
# last_week_cols=['ZIP']
# for i in range(8):
#     last_week_cols.append(all_dates.columns.to_list()[-8:][i])
# last_week=all_dates[last_week_cols]
# last_week.to_csv('last_week.csv')
# item_prop = {'title':'last_week_data'}

# csv_item = gis.content.add(item_properties=item_prop, data='last_week.csv')
# csv_item.publish()

In [33]:
all_dates=pd.DataFrame.spatial.from_layer(zip_point_base_fl.layers[0])
last_week_cols=['ZIP']
for i in range(9):
    last_week_cols.append(all_dates.columns.to_list()[-9:][i])
last_week=all_dates[last_week_cols]

last_week_name='last_week'+'.csv'
last_week.to_csv(last_week_name,index=False)
last_week_ly=gis.content.get('c681ad16489e4a6599ab773858684680')
last_week_lyr=last_week_ly.layers[0]

In [18]:
# Delete the passed dates

for i in last_week_lyr.properties.fields:
    if i['name'] not in ['ZIP','ObjectId']:
        try:
            to_be_delete={'fields':
                              [
                                  {
                                      'name': i['name']
                                  }
                              ]
                         }
            last_week_lyr.manager.delete_from_definition(to_be_delete)
        except:
            continue

In [19]:
# Add new last week dates 
last_week_add=[]
minus=['GlobalID','SHAPE','ZIP']
last_week_field_list=[]
new_last_week_dates=[]


for i in last_week.columns:
    if i not in minus:
        new_last_week_dates.append(i)
        dols_field = dict(deepcopy(zip_point_lyr.properties.fields[1]))
        dols_field['name'] = i
        dols_field['alias'] = i.replace('F','')
        dols_field['type'] = "esriFieldTypeDouble"
        last_week_field_list.append(dols_field)
    else:
        continue
last_week_lyr.manager.add_to_definition({"fields":last_week_field_list})

{'success': True}

In [35]:
#Append data
for i in new_last_week_dates:
    i=i.replace('F','')
    new='F'+i
    last_week_lyr.append(item_id=csv_wide_item.id,
                          upload_format = 'csv',
                          field_mappings = [{"name":new, "source":i},
                                            {"name":"ZIP", "source":"zipcode"}],
                          source_info = zip_wide_csv_info['publishParameters'],
                          upsert=True,
                          update_geometry=False,
                          append_fields=[new, "ZIP"],
                          skip_inserts=True,
                          upsert_matching_field="ZIP")
    print('done for '+ i)

last_week_lyr.append(item_id=csv_wide_item.id,
                          upload_format = 'csv',
                          field_mappings = [{"name":'Current_Cases', "source":new_dates[-1].replace('F','')},
                                            {"name":"ZIP", "source":"zipcode"}],
                          source_info = zip_wide_csv_info['publishParameters'],
                          upsert=True,
                          update_geometry=False,
                          append_fields=['Current_Cases', "ZIP"],
                          skip_inserts=True,
                          upsert_matching_field="ZIP")

True

## 6. Update the current cases count in predicted map

In [39]:
prely=gis.content.get('ce9ca208c171444bbdda0ec54f5b6a07')
pre=pd.DataFrame.spatial.from_layer(demoly.layers[0])

prely.layers[0].append(item_id=csv_wide_item.id,
                          upload_format = 'csv',
                          field_mappings = [{"name":'Current_Cases', "source":new_dates[-1].replace('F','')},
                                            {"name":"ZIP", "source":"zipcode"}],
                          source_info = zip_wide_csv_info['publishParameters'],
                          upsert=True,
                          update_geometry=False,
                          append_fields=['Current_Cases', "ZIP"],
                          skip_inserts=True,
                          upsert_matching_field="ZIP")

True